In [65]:
###############################################################################################################################
#MO Dashboard Tool v2
###############################################################################################################################
#The purpose of this tool is to automate the filling process of the MO Dashboard. 
#To run this tool in Jupyter Notebook, left click "Cell" at the top and left click "Run All". A console should appear in which
#you can begin the filling process.
#
##################################################################################################
#Versions:                                                                                       #
#v1: Tool is created                                                                             #
#v2: Captures Work Refusals data and developed functionality of recursive search for raw data.   #
#                                                                                                #
##################################################################################################

#Make sure that the file code to the relevant files are correct 
file_dict = {'Job Seeker Status Zone' : 'REG18-0273',
             'Daily and Cumulative Demerits' : 'REG20-0321',
             'Suspension Events' : 'REG20-0430',
             'Payment Suspensions' : 'REG18-0272',
             'Work Refusal' : 'REG18-0275'}
###############################################################################################################################

In [66]:
#Import statements
import openpyxl
from openpyxl.styles import PatternFill, Font, Border, Side, Alignment
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.drawing.image import Image
import pandas as pd
import numpy as np
import os
import glob
import re
import tkinter as tk
from tkinter.filedialog import askopenfilename, askdirectory
from tkinter import messagebox, Checkbutton

#Establish borders
medium = Side(border_style="medium", color="000000")
thin = Side(border_style="thin", color="000000")

#Load image file (for the traffic light)
img = Image(r'H:\JOB SEEKER COMPLIANCE TEAM 1 JULY 2017\Data\Data\Data From DSS\Dashboard etc\Images\trafficlight.png')

In [67]:
#Define Utility Functions
def set_border(ws, cell_range):
    """
    Purpose: To put a medium bolded border around the specified area (eg. "A1:B3") in the excel spreadsheet
    
    Function Variables:
        ws: worksheet object
        cell_range: range in the excel worksheet in which you would like to put a border on.
    """
    rows = list(ws[cell_range])
    side = Side(border_style='medium', color="FF000000")

    rows = list(rows)  # we convert iterator to list for simplicity
    max_y = len(rows) - 1  # index of the last row
    for pos_y, cells in enumerate(rows):
        max_x = len(cells) - 1  # index of the last cell
        for pos_x, cell in enumerate(cells):
            border = Border(
                left=cell.border.left,
                right=cell.border.right,
                top=cell.border.top,
                bottom=cell.border.bottom
            )
            if pos_x == 0:
                border.left = side
            if pos_x == max_x:
               border.right = side
            if pos_y == 0:
               border.top = side
            if pos_y == max_y:
               border.bottom = side

            # set new border only if it's one of the edge cells
            if pos_x == 0 or pos_x == max_x or pos_y == 0 or pos_y == max_y:
               cell.border = border
            
def reorder_list(list_to_order, reorder_list):
    """
    Purpose: To reorder a list of items based on the reorder_list. This is due to categories being in a different order in the 
             raw data as opposed to the order in the final dashboard.
    
    Function Variables:
        list_to_order: list of items to order
        reorder_list: the order in which you would like list_to_order to be re-ordered
    """    
    list_to_order[:] = [list_to_order[i] for i in reorder_list]
    
def load_template_workbook():
    """
    Purpose: to load the dashboard template which will be used to create the final dashboard.
    
    Methodology:
        Searches for 'Master Template' sheet in the template excel file to be used as the template. Make sure the template has
        'Master Template' as one of the worksheets.
    """  
    global wb, ws
    #Loading template
    wb = openpyxl.load_workbook(template_path)
    ws = wb['Master Template']
    ws.title = 'MO Dashboard'

def get_file_recursively(file_path, document_code):
    """
    Purpose: function developed to search for files recursiverly in a parent folder and all it's sub folders.
    
    Methodology:
        Searches for the file based on the document code and the date time code. The document code is provided to the function
        and the datetime_code is the conversion of the date inputted into the GUI console in a "01062021" format. This is due
        to the naming convention of the files that we receive from SA. 
        
        It expects the files to be named {document code} ... {datetime code}.
        
        The function will first search specifically for the folder based on the standardised folder structure, if that fails,
        if will search recursively through the parent folder and all sub-folders for the file.
    """
    global report_period
    datetime_code = '{:02d}{:02d}{:04d}'.format(pd.to_datetime(report_period).day, pd.to_datetime(report_period).month, pd.to_datetime(report_period).year)
    try:
        #Tool will attempt to search through the folder structure based on the convention: REG18-0273 - Job Seeker Status Zone\202107 - July
        standardised_file_path = glob.glob(glob.glob(glob.glob(file_path + r'\{}*'.format(document_code))[0] + r'\{:04d}{:02d}*'.format(pd.to_datetime(report_period).year, pd.to_datetime(report_period).month))[0] + r'\{}*{}.xlsx'.format(document_code, datetime_code))[0]
        return standardised_file_path
    except:
        if recursive_search_flag == 1: #Perform recursive search if standardised search fails and the check box has been checked by the user     
            recursive_file_path = next(glob.iglob(file_path + r'\**\{}*{}.xlsx'.format(document_code, datetime_code), recursive = True))
            return recursive_file_path
            
class ImportFile:
    """
    Purpose: To import the excel files which can be used for filling of the MO Dashboards
    
    Class Variables:
        file_path: path of the file name
        sheet_name: name of the sheet
        document_code: code of the file (eg. REG18-0273)
        wb
        sheet
        value_to_find: value to be found to assist with the skiprows parameter
    
    Class Methods:
        get_actual_file_path: For situations where a parent level folder is selected and the appropriate files are in the
                              subfolders, this method will search for the appropriate files recursively.
        find_row_exact: Finds the cell row (by searching for the cell that contains exactly value_to_find) for the skiprows 
                        parameter which will allow the file to be imported correctly
        find_row_regex: Finds the cell row (by searching for the cell that contains the regex value_to_find) for the skiprows 
                        parameter which will allow the file to be imported correctly
        import_file: imports the file and returns it as a pandas dataframe using find_row_exact (if not None) otherwise, uses 
                     find_row_regex for the skiprows parameter
    """
    def __init__(self, file_path, sheet_name, document_code, value_to_find):
        self.file_path = file_path
        self.sheet_name = sheet_name
        self.document_code = document_code
        self.value_to_find = value_to_find
    
    @property
    def wb(self):
        return openpyxl.load_workbook(filename = self.file_path)
    @property
    def sheet(self):
        return self.wb[self.sheet_name]
    
    def get_actual_file_path(self):
        self.file_path = get_file_recursively(self.file_path, self.document_code)
    
    def find_row_exact(self):
        for row in self.sheet:
            for cell in row:
                if cell.value == self.value_to_find:
                    return cell.row
    def find_row_regex(self):
        for row in self.sheet:
            for cell in row:
                if re.match(self.value_to_find, str(cell.value)):
                    return cell.row
    def import_file(self, rows_to_skip):
        if self.find_row_exact() is not None:
            return pd.read_excel(self.file_path,
                                   skiprows = self.find_row_exact() + rows_to_skip, # Add rows to the found row due to the design of the report
                                   sheet_name = self.sheet_name)
        else:
            return pd.read_excel(self.file_path,
                                   skiprows = self.find_row_regex() + rows_to_skip, # Add rows to the found row due to the design of the report
                                   sheet_name = self.sheet_name)

In [68]:
###############################################################################################################################
#Import file process function
###############################################################################################################################
def import_files():
    global data18_0273, data20_0321, data20_0430, data18_0272, data18_0275, data18_0275_SA, data18_0275_Excuses
    #Import files
    
    #Search for 'Job seekers by Status Zone'
    data18_0273 = ImportFile(files_path, 'Summary Tables', file_dict['Job Seeker Status Zone'], 'Job seekers by Status Zone')
    data18_0273.get_actual_file_path()
    data18_0273 = data18_0273.import_file(rows_to_skip = 2)
    
    #Search for 'Table 3 - Current Confirmed Demerits'
    data20_0321 = ImportFile(files_path, 'Summary with Digital', file_dict['Daily and Cumulative Demerits'], 'Table 3 - Current Confirmed Demerits')
    data20_0321.get_actual_file_path()
    data20_0321 = data20_0321.import_file(rows_to_skip = 3)
    
    #Search for 'Table 3' at the start of the cell
    data20_0430 = ImportFile(files_path, 'Summary', file_dict['Suspension Events'], r'^Table 3')
    data20_0430.get_actual_file_path()
    data20_0430 = data20_0430.import_file(rows_to_skip = 3)
    
    #Search for 'Table 3 - Cumulative Payment suspensions'
    data18_0272 = ImportFile(files_path, 'From 28 Sept', file_dict['Payment Suspensions'], 'Table 3 - Cumulative Payment suspensions')
    data18_0272.get_actual_file_path()
    data18_0272 = data18_0272.import_file(rows_to_skip = 2)
    
    #Search for 'Table 2 - Work Refusal Reports submitted to Services Australia by State'
    data18_0275 = ImportFile(files_path, 'Jobactive WRR from 4 Aug', file_dict['Work Refusal'], 'Table 2 - Work Refusal Reports submitted to Services Australia by State')
    data18_0275.get_actual_file_path()
    data18_0275 = data18_0275.import_file(rows_to_skip = 1)
    
    #Search for 'Table 1 - Current status of Work Refusal Reports submitted to Services Australia'
    data18_0275_SA = ImportFile(files_path, 'Jobactive WRR from 4 Aug', file_dict['Work Refusal'], 'Table 1 - Current status of Work Refusal Reports submitted to Services Australia')
    data18_0275_SA.get_actual_file_path()
    data18_0275_SA = data18_0275_SA.import_file(rows_to_skip = 1)
    
    #Search for 'Table 4 - Top 3 reasons a Reasonable Excuse was accepted by Services Australia'
    data18_0275_Excuses = ImportFile(files_path, 'Jobactive WRR from 4 Aug', file_dict['Work Refusal'], 'Table 4 - Top 3 reasons a Reasonable Excuse was accepted by Services Australia')
    data18_0275_Excuses.get_actual_file_path()
    data18_0275_Excuses = data18_0275_Excuses.import_file(rows_to_skip = 1)

In [69]:
###############################################################################################################################
#Top 3 Reasonable Excuse (jobactive/Digital & DES)
###############################################################################################################################
def top_three_reasonable_excuse():
    global ws
    #Search and set index
    excuses_index = data18_0275_Excuses.columns.get_loc('Reasonable Excuse reason')
    totals_index = data18_0275_Excuses.columns.get_loc('Total')
    excuses_DES_index = data18_0275_Excuses.columns.get_loc('Reasonable Excuse reason.1')
    totals_DES_index = data18_0275_Excuses.columns.get_loc('Total.1')

    excuses = data18_0275_Excuses.iloc[0:3, excuses_index].tolist()
    totals = data18_0275_Excuses.iloc[0:3, totals_index].tolist()

    excuses_DES = data18_0275_Excuses.iloc[0:3, excuses_DES_index].tolist()
    totals_DES = data18_0275_Excuses.iloc[0:3, totals_DES_index].tolist()

    #Insert into a dictionary
    value_dict = {
                    'excuses':excuses,
                    'totals':totals,
                    'excuses_DES':excuses_DES,
                    'totals_DES':totals_DES
                    }

    #Fill jobactive/Digital and DES values
    column_iterative_index = np.linspace(15,24,4) #Column increments of 3 for each column (including jobactive/Digital & DES)
    row_iterative_index = np.linspace(61,65,3) # Increments of 2 for each row fill
    for col_index, dict_index in zip(column_iterative_index, list(value_dict.keys())):
        for r_index, value in zip(row_iterative_index, value_dict[dict_index]):
            ws.cell(row = int(r_index), column = col_index, value = value)

    #Establish borders
    #jobactive/Digital table
    #Set thin borders
    for rows in ws.iter_rows(min_row=57, min_col=15, max_row=66, max_col=19):
        for cell in rows:
            cell.border = Border(top = thin, left = thin, right = thin, bottom = thin)
    set_border(ws, 'O57:S66')

    #DES table
    #Set thin borders
    for rows in ws.iter_rows(min_row=57, min_col=21, max_row=66, max_col=25):
        for cell in rows:
            cell.border = Border(top = thin, left = thin, right = thin, bottom = thin)
    set_border(ws, 'U57:Y66')

In [70]:
###############################################################################################################################
#Work Refusals table (to SA)
###############################################################################################################################
def work_refusal_table_SA():
    global ws
    #Search and set index
    job_active_index = data18_0275_SA.columns.get_loc('jobactive')
    digital_index = data18_0275_SA.columns.get_loc('Digital')
    des_index = data18_0275_SA.columns.get_loc('DES')

    #Extract jobactive
    jobactive = data18_0275_SA.iloc[0:3, job_active_index].tolist()
    #Extract Digital
    digital = data18_0275_SA.iloc[0:3, digital_index].tolist()
    #Extract DES
    DES = data18_0275_SA.iloc[0:3, des_index].tolist()
    #Calculate Totals
    Total = np.sum([jobactive, digital, DES], axis = 0)

    #Insert into a dictionary
    value_dict = {
                'jobactive':jobactive,
                'digital':digital,
                'DES':DES,
                'Total':Total
                }
    #Fill values
    for dict_index, cols in enumerate(ws.iter_cols(min_row=61, min_col=10, max_row=63, max_col=13)):
        dict_key_to_use = list(value_dict.keys())[dict_index]
        fill_list = value_dict[dict_key_to_use]
        for list_index, cell in enumerate(cols):
            cell.value = fill_list[list_index]
            cell.font = Font(name = 'Gill Sans MT', size = 10, bold = False, italic = False, color = 'FF000000')
    #Fill totals
    for row in ws.iter_rows(min_row=64, min_col=10, max_row=64, max_col=13):
        for index, cell in enumerate(row):
                dict_key_to_use = list(value_dict.keys())[index]
                fill_list = value_dict[dict_key_to_use]
                cell.value = np.sum(fill_list) #Sum of all values
                cell.font = Font(name = 'Gill Sans MT', size = 10, bold = True, italic = False, color = 'FF000000')  
    #Format borders
    for rows in ws.iter_rows(min_row=59, min_col=7, max_row=64, max_col=13):
        for cell in rows:
            cell.border = Border(top = thin, left = thin, right = thin, bottom = thin)
    set_border(ws, 'G57:M64')
    set_border(ws, 'G57:M58')

    #Format bold font for totals column
    for cols in ws.iter_cols(min_row=61, min_col=13, max_row=64, max_col=13):
        for cell in cols:
            cell.font = Font(name = 'Gill Sans MT', size = 10, bold = True, italic = False, color = 'FF000000')

In [71]:
###############################################################################################################################
#Work Refusals table (By State)
###############################################################################################################################
def work_refusal_table_STATE():
    global ws
    #Search and set index
    job_active_index = data18_0275.columns.get_loc('jobactive')
    digital_index = data18_0275.columns.get_loc('Digital')
    des_index = data18_0275.columns.get_loc('DES')

    #Extract jobactive
    jobactive = data18_0275.iloc[0:8, job_active_index].tolist()
    #Extract Digital
    digital = data18_0275.iloc[0:8, digital_index].tolist()
    #Extract DES
    DES = data18_0275.iloc[0:8, des_index].tolist()
    #Calculate Totals
    Total = np.sum([jobactive, digital, DES], axis = 0)

    #Insert into a dictionary
    value_dict = {
                'jobactive':jobactive,
                'digital':digital,
                'DES':DES,
                'Total':Total
                }

    #Fill values
    for dict_index, cols in enumerate(ws.iter_cols(min_row=60, min_col=2, max_row=67, max_col=5)):
        dict_key_to_use = list(value_dict.keys())[dict_index]
        fill_list = value_dict[dict_key_to_use]
        for list_index, cell in enumerate(cols):
            cell.value = fill_list[list_index]
            cell.font = Font(name = 'Gill Sans MT', size = 10, bold = False, italic = False, color = 'FF000000')
    #Fill totals
    for row in ws.iter_rows(min_row=68, min_col=2, max_row=68, max_col=5):
        for index, cell in enumerate(row):
                dict_key_to_use = list(value_dict.keys())[index]
                fill_list = value_dict[dict_key_to_use]
                cell.value = np.sum(fill_list) #Sum of all values
                cell.font = Font(name = 'Gill Sans MT', size = 10, bold = True, italic = False, color = 'FF000000')        
    #Set border
    for rows in ws.iter_rows(min_row=57, min_col=1, max_row=57, max_col=5):
        for index, cell in enumerate(rows):
            if index == 0:
                cell.border = Border(top = medium, left = medium)
            elif index != 4:
                cell.border = Border(top = medium)
            else:
                cell.border = Border(top = medium, right = medium)  
    for rows in ws.iter_rows(min_row=58, min_col=1, max_row=58, max_col=5):
        for index, cell in enumerate(rows):
            if index == 0:
                cell.border = Border(bottom = medium, left = medium)
            elif index != 4:
                cell.border = Border(bottom = medium)
            else:
                cell.border = Border(bottom = medium, right = medium)
    #Bold total column
    for cols in ws.iter_cols(min_row=60, min_col=5, max_row=67, max_col=5):
        for cell in cols:
            cell.font = Font(name = 'Gill Sans MT', size = 10, bold = True, italic = False, color = 'FF000000')  

In [72]:
###############################################################################################################################
#Payment Suspensions table
###############################################################################################################################
def payment_suspensions_table():
    global ws, job_active_jscount, digital_jscount, des_jscount, pn_jscount
    #Search and set index
    job_active_index = data18_0272.columns.get_loc('jobactive - excludes digital')
    digital_index = data18_0272.columns.get_loc('Digital')
    des_index = data18_0272.columns.get_loc('DES')
    pn_index = data18_0272.columns.get_loc('ParentsNext')
    total_index = data18_0272.columns.get_loc('Total')

    #Create
    job_active_total = data18_0272.iloc[2:13, job_active_index].tolist()
    job_active_jscount = data18_0272.iloc[2:13, job_active_index+1].tolist()
    
    digital_total = data18_0272.iloc[2:13, digital_index].tolist()
    digital_jscount = data18_0272.iloc[2:13, digital_index+1].tolist()

    des_total = data18_0272.iloc[2:13, des_index].tolist()
    des_total[len(des_total)-3] = np.nan #Remove Flexible Learning and Points Based Period
    des_jscount = data18_0272.iloc[2:13, des_index+1].tolist()
    des_jscount[len(des_total)-3] = np.nan #Remove Flexible Learning and Points Based Period

    pn_total = data18_0272.iloc[2:13, pn_index].tolist()
    pn_total[len(pn_total)-3] = np.nan #Remove Flexible Learning and Points Based Period
    pn_jscount = data18_0272.iloc[2:13, pn_index+1].tolist()
    pn_jscount[len(pn_jscount)-3] = np.nan #Remove Flexible Learning and Points Based Period

    total_total = data18_0272.iloc[2:13, total_index].tolist()
    total_jscount = data18_0272.iloc[2:13, total_index+1].tolist()

    #Insert into a dictionary
    value_dict = {
                'job_active_total':job_active_total,
                'job_active_jscount':job_active_jscount,
                'digital_total':digital_total,
                'digital_jscount':digital_jscount,
                'des_total':des_total,
                'des_jscount':des_jscount,
                'pn_total':pn_total,
                'pn_jscount':pn_jscount,
                'total_total':total_total,
                'total_jscount':total_jscount
                }
    #Fill values
    for dict_index, cols in enumerate(ws.iter_cols(min_row=43, min_col=4, max_row=53, max_col=13)):
        dict_key_to_use = list(value_dict.keys())[dict_index]
        fill_list = value_dict[dict_key_to_use]
        for list_index, cell in enumerate(cols):
            cell.value = fill_list[list_index]
            cell.font = Font(name = 'Gill Sans MT', size = 10, bold = False, italic = False, color = 'FF000000')

    #Set thin bottom borders on the left panel
    for cols in ws.iter_cols(min_row=40, min_col=2, max_row=52, max_col=3):
        for cell in cols:
            cell.border = Border(bottom = thin)
    #Set bold borders
    #Side border
    for rows in ws.iter_rows(min_row=40, min_col=2, max_row=42, max_col=13):
        for cell in rows:
            cell.border = Border(left = medium, right = medium, top = medium, bottom = thin)
    for rows in ws.iter_rows(min_row=42, min_col=12, max_row=52, max_col=12):
        for cell in rows:
            cell.border = Border(left = medium, bottom = thin, top = thin, right = thin)
    ws['H52'].border = Border(left = medium, bottom = thin, top = thin, right = medium)
    ws['J52'].border = Border(left = medium, bottom = thin, top = thin, right = medium)
    #Outer border
    set_border(ws, 'B39:M53')

In [73]:
###############################################################################################################################
#Suspension Events table
###############################################################################################################################
def suspension_events_table():
    global ws
    #Fill values
    rows = dataframe_to_rows(data20_0430.iloc[0:4,1:6], index = False, header = False)
    iterative_index = np.linspace(0,6,4) # Increments of 2 for each row fill
    for r_index, row in zip(iterative_index, rows):
        for c_index, value in enumerate(row):
            ws.cell(row = int(r_index) + 22, column = c_index + 12, value = value) #Start from row 22 and column 12

    #Format borders Demerits table    
    #Set thin borders
    for rows in ws.iter_rows(min_row=22, min_col=10, max_row=27, max_col=16):
        for cell in rows:
            cell.border = Border(top = thin, left = thin, right = thin, bottom = thin)
    #Set total border
    for rows in ws.iter_rows(min_row=28, min_col=10, max_row=28, max_col=16):
        for cell in rows:
            cell.border = Border(top = medium, left = medium, right = medium, bottom = medium)
    set_border(ws, 'J17:P20')
    set_border(ws, 'J17:P28')

In [74]:
###############################################################################################################################
#Demerits Applied - Cumulative table
###############################################################################################################################
def demerits_applied_cumulative_table():
    global ws
    #Extract Stream A
    Stream_A = data20_0321.iloc[0:9, 1].tolist()
    #Extract Stream B
    Stream_B = data20_0321.iloc[0:9, 2].tolist()
    #Extract Stream C
    Stream_C = data20_0321.iloc[0:9, 3].tolist()
    #Extract Digitally Serviced
    Digital = data20_0321.iloc[0:9, 4].tolist()
    #Extract DES
    DES = data20_0321.iloc[0:9, 5].tolist()
    DES[len(DES)-2:len(DES)] = [0, 0] #Remove Flexible Learning and Points Based Period
    #Extract Parents Next
    PN = data20_0321.iloc[0:9, 6].tolist()
    PN[len(PN)-2:len(PN)] = [0, 0] #Remove Flexible Learning and Points Based Period
    #Calculate Totals
    Total = np.sum([Stream_A, Stream_B, Stream_C, Digital, DES, PN], axis = 0)

    #Convert to null values for the purpose of reporting
    PN[len(PN)-2:len(PN)] = [np.nan, np.nan] #Remove Flexible Learning and Points Based Period
    DES[len(DES)-2:len(DES)] = [np.nan, np.nan] #Remove Flexible Learning and Points Based Period

    #Insert into a dictionary
    value_dict = {
                'Stream A':Stream_A,
                'Stream B':Stream_B,
                'Stream C':Stream_C,
                'Digital':Digital,
                'DES':DES,
                'PN':PN,
                'Total':Total,
                }
    #Fill values
    for dict_index, cols in enumerate(ws.iter_cols(min_row=22, min_col=2, max_row=30, max_col=8)):
        dict_key_to_use = list(value_dict.keys())[dict_index]
        fill_list = value_dict[dict_key_to_use]
        for list_index, cell in enumerate(cols):
            cell.value = fill_list[list_index]
            cell.font = Font(name = 'Gill Sans MT', size = 10, bold = False, italic = False, color = 'FF000000')
            cell.fill = PatternFill(fgColor="FFFFFF", fill_type = "solid")
            cell.alignment = Alignment(horizontal = 'center', vertical = 'center')

    #Fill totals
    for row in ws['B31':'H31']:
        for index, cell in enumerate(row):
                dict_key_to_use = list(value_dict.keys())[index]
                fill_list = value_dict[dict_key_to_use]
                cell.value = np.nansum(fill_list) #Sum of all values
                cell.font = Font(name = 'Gill Sans MT', size = 10, bold = True, italic = False, color = 'FF000000')

    #Fill jobactive total
    ws['B32'] = np.sum([value_dict['Stream A'], value_dict['Stream B'], value_dict['Stream C'], value_dict['Digital']])


    #Format borders in table
    #Format borders Demerits table
    for rows in ws.iter_rows(min_row=17, min_col=1, max_row=21, max_col=8):
        for cell in rows:
            cell.border = Border(top = medium, left = medium, right = medium, bottom = medium)

    #Format border in Demerits table (jobactive total)
    for rows in ws.iter_rows(min_row=32, min_col=1, max_row=32, max_col=5):
        for cell in rows:
            cell.border = Border(top = medium, left = medium, right = medium, bottom = medium)

    #Bold total column
    for cols in ws.iter_cols(min_row = 22, min_col = 8, max_row = 31, max_col = 8):
        for cell in cols:
            cell.font = Font(name = 'Gill Sans MT', size = 10, bold = True, italic = False, color = 'FF000000')

    #Grey out Flexible Learning & Points Based Period (for DES & Parents Next)
    for cols in ws.iter_cols(min_row = 29, min_col = 6, max_row = 30, max_col = 7):
        for cell in cols:
            cell.fill = PatternFill(fgColor="A9A9A9", fill_type = "solid")

In [75]:
###############################################################################################################################
#Job Seekers by Zone table
###############################################################################################################################
def zone_table():
    global ws, value_dict
    #Initate list order required
    required_list_order = ['jobactive', 'Digital', 'DES', 'ParentsNext']
    reindex_list = []
    for i in required_list_order:
        index_found = data18_0273.loc[~pd.isna(data18_0273['Programme']), 'Programme'].tolist().index(i)
        reindex_list.append(index_found)

    #Extract green zone numbers
    green_zone = data18_0273.iloc[2:6, 2].tolist()
    #Extract warning zone numbers
    one_demerit = data18_0273.iloc[2:6, 4].tolist()
    two_demerit = data18_0273.iloc[2:6, 5].tolist()
    three_demerit = data18_0273.iloc[2:6, 6].tolist()
    four_demerit = data18_0273.iloc[2:6, 7].tolist()
    five_demerit = data18_0273.iloc[2:6, 8].tolist()
    #Extract all zone numbers
    zero_penalty = data18_0273.iloc[2:6, 10].tolist()
    one_penalty = data18_0273.iloc[2:6, 12].tolist()
    two_penalty = data18_0273.iloc[2:6, 13].tolist()
    three_penalty = data18_0273.iloc[2:6, 14].tolist()

    #Reorder lists
    lists_to_reorder = [green_zone, one_demerit, two_demerit, three_demerit, 
                        four_demerit, five_demerit, zero_penalty, one_penalty, 
                        two_penalty, three_penalty]
    for i in lists_to_reorder:
        reorder_list(i, reindex_list)

    #Calculate total warning & penalty numbers
    total_warning = np.sum([one_demerit, two_demerit, three_demerit, four_demerit, five_demerit], axis = 0)
    total_penalty = np.sum([zero_penalty, one_penalty, two_penalty, three_penalty], axis = 0)
    #Calculate all zone numbers
    all_zones = np.sum([green_zone, total_warning, total_penalty], axis = 0)

    #Insert into a dictionary
    value_dict = {
                'All Zones':all_zones,
                'Green Zone':green_zone,
                'Total Warning Zone':total_warning,
                '1 Demerit Zone':one_demerit,
                '2 Demerit Zone':two_demerit,
                '3 Demerit Zone':three_demerit,
                '4 Demerit Zone':four_demerit,
                '5 Demerit Zone':five_demerit,
                '0 Penalty Zone':zero_penalty,
                '1 Penalty Zone':one_penalty,
                '2 Penalty Zone':two_penalty,
                '3 Penalty Zone':three_penalty,
                'Total Penalty':total_penalty
                }

    #Fill values
    for dict_index, cols in enumerate(ws.iter_cols(min_row=8, min_col=2, max_row=11, max_col=14)):
        dict_key_to_use = list(value_dict.keys())[dict_index]
        fill_list = value_dict[dict_key_to_use]
        for list_index, cell in enumerate(cols):
            cell.value = fill_list[list_index]
            cell.font = Font(name = 'Gill Sans MT', size = 10, bold = False, italic = False, color = 'FF000000')
            cell.fill = PatternFill(fgColor="FFFFFF", fill_type = "solid")
            cell.alignment = Alignment(horizontal = 'center', vertical = 'center')

    #Fill Total
    color_list = ['809ec2'] + ['8fb08c'] + ['f5cd2d' for i in range(0,6)] + ['FF0000' for i in range(0,7)]
    for row in ws['B12':'N12']:
        for index, cell in enumerate(row):
                dict_key_to_use = list(value_dict.keys())[index]
                fill_list = value_dict[dict_key_to_use]
                cell.value = np.sum(fill_list) #Sum of all values
                cell.font = Font(name = 'Gill Sans MT', size = 10, bold = False, italic = False, color = 'FF000000')
                cell.fill = PatternFill(fgColor=color_list[index], fill_type = "solid")
                cell.alignment = Alignment(horizontal = 'center', vertical = 'center')
    #Fill job active
    for row in ws['B7':'N7']:
        for index, cell in enumerate(row):
                dict_key_to_use = list(value_dict.keys())[index]
                fill_list = value_dict[dict_key_to_use]
                cell.value = np.sum(fill_list[0:2]) #Sum of provider manage and digitally serviced
                cell.font = Font(name = 'Gill Sans MT', size = 10, bold = False, italic = False, color = 'FF000000')
                cell.fill = PatternFill(fgColor="FFFFFF", fill_type = "solid")
                cell.alignment = Alignment(horizontal = 'center', vertical = 'center')
    #Fill Total in Zones (traffic light)
    fill_index = ['Green Zone', 'Green Zone', 'Total Warning Zone','Total Warning Zone', 'Total Warning Zone','Total Penalty','Total Penalty']
    for index, col in enumerate(ws['Q5:Q11']):
        for cell in col:
            cell.value = np.round(np.sum(value_dict[fill_index[index]])/np.sum(value_dict['All Zones']), 4)
    ws['Q4'] = np.sum(value_dict['All Zones'])

    #Format borders in table

    #Format borders in Jobseeker by Zone table
    for rows in ws.iter_rows(min_row=4, min_col=1, max_row=6, max_col=14):
        for cell in rows:
            cell.border = Border(top = medium, left = medium, right = medium, bottom = medium)

    #Format borders in traffic light
    for cols in ws.iter_cols(min_row=5, min_col=17, max_row=11, max_col=17):
        for cell in cols:
            cell.border = Border(top = thin, left = thin, right = thin, bottom = thin)
    #Insert traffic light image
    ws.add_image(img, 'P5')
    ws.merge_cells(start_row=5, start_column=16, end_row=11, end_column=16)
    #Set final border around traffic light table
    set_border(ws, 'P4:Q11')

    #Fix bold formats in Program index 
    for col in ws['A8':'A9']:
        for cell in col:
            cell.font = Font(name = 'Gill Sans MT', size = 10, bold = False)

    #Bold the total row
    color_list = ['FF000000' for i in range(0,9)] + ['FFFFFF' for i in range(0,6)]
    for rows in ws.iter_rows(min_row=12, min_col=1, max_row=12, max_col=14):
        for index, cell in enumerate(rows):
            cell.font = Font(name = 'Gill Sans MT', size = 10, bold = True, color = color_list[index])

In [76]:
###############################################################################################################################
#Fill Key Stats
###############################################################################################################################
def key_stats():
    global ws, value_dict
    stats_list = [ 
                    '  {:,} job seekers had received at least one payment suspension since 28 September 2020',
                    '  {:,} job seekers are in the warning zone, of which:',
                    '     - {:,} have one demerit',
                    '     - {:,} have two demerits',
                    '     - {:,} have three demerits',
                    '     - {:,} have four demerits',
                    '     - {:,} have five demerits',
                    '  {:,} job seekers are in the Penalty Zone'
                ]
    fill_index = ['Total Warning Zone', '1 Demerit Zone', '2 Demerit Zone', 
                  '3 Demerit Zone', '4 Demerit Zone', '5 Demerit Zone', 
                  'Total Penalty']
    #Initiate fill
    fill_counter = 0
    for i in np.linspace(6,36,4): #Loop through
        for cols in ws.iter_cols(min_row=int(i), min_col=19, max_row=int(i)+7, max_col=19):
            for index, cell in enumerate(cols):
                if index != 0:
                    cell.value = stats_list[index].format(int(value_dict[fill_index[index-1]][fill_counter]))
                else:
                    pass #To be filled at (Fill Key Stats Job Seeker count)
        fill_counter += 1
    set_border(ws, 'S4:Z43')

    ###############################################################################################################################
    #Fill Key Stats Job Seeker count
    ###############################################################################################################################
    fill_list = [job_active_jscount, digital_jscount, des_jscount, pn_jscount]
    for index, i in zip(np.linspace(6,36,4), fill_list):
        ws.cell(row = int(index), column = 19).value = stats_list[0].format(i[-1])
    ###############################################################################################################################
    #Fill Key Stats High Level Count
    ###############################################################################################################################
    fill_list = value_dict['All Zones']
    stats_list = [
                    'jobactive - provider managed ({:,})',
                    'Digitally Serviced ({:,})',
                    'DES ({:,})',
                    'ParentsNext ({:,})'
                ]
    for stats_index, (index, value) in enumerate(zip(np.linspace(5,35,4), fill_list)):
        ws.cell(row = int(index), column = 19).value = stats_list[stats_index].format(int(value))

In [77]:
#Additional Add-Ons
def additional_add_ons():
    global ws
    ws['A2'].value = 'Return to Mutual Obligation Requirements - Point in time - {}'.format(report_period) #Set subtitle
    ws.row_dimensions[63].height = 25 #Adjust height of row 62

In [78]:
#Extraction
def output_workbook():
    global wb, ws
    list_of_sheets = wb.sheetnames
    list_of_sheets.remove(ws.title) #Remove every other worksheet besides the final dashboard
    for sheet in list_of_sheets:
        wb.remove(wb[sheet])
    wb.save(output_path + '\Dashboard 15 - MO Targeted Compliance Framework Dashboard {}.xlsx'.format(report_period))

In [79]:
#Console Functions
def load_and_fill_workbook():
    global recursive_search_flag
    recursive_search_flag = var.get()
    statusBar.config(text='Filling in progress...')
    root.update_idletasks()    
    errmsg = None
    try:
        load_template_workbook()
        import_files()
    except:
        errmsg = 'File error, make sure input/template files are closed! Please check the specified template file and the files path are correct!'
    try:
        top_three_reasonable_excuse()
    except:
        errmsg = 'Filling error, please check the top 3 reasonable excuse table input data!'
    try:
        work_refusal_table_STATE()
        work_refusal_table_SA()
    except:
        errmsg = 'Filling error, please check the work refusals data!'
    try:
        payment_suspensions_table()
    except:
        errmsg = 'Filling error, please check the payment suspensions data!'
    try:
        suspension_events_table()
    except:
        errmsg = 'Filling error, please check the suspensions events data!'
    try:
        demerits_applied_cumulative_table()
    except:
        errmsg = 'Filling error, please check the work refusals data!'
    try:
        zone_table()
        key_stats()
    except:
        errmsg = 'Filling error, please check the zones data!'
    try:
        additional_add_ons()
        output_workbook()
    except:
        errmsg = 'Output error, please check the output path!'
    
    if errmsg is None:
        tk.messagebox.showinfo('MO Dashboard Tool', 'Dashboard has been filled and uploaded to the output path!')
        inputBtn_master.config(state='disabled')
        inputBtn_files.config(state='disabled')
        inputBtn_output.config(state='disabled')
        startBtn.config(state='disabled')
        checkbox_files.config(state='disabled')
    else:
        tk.messagebox.showinfo('MO Dashboard Tool', errmsg)

    statusBar.config(text='Console Ready!\nEnter the Reporting Period.')
    #Reset entry bar and input buttons
    entrybar_period.config(state='normal')
    inputBtn_period.config(state='normal')
    root.update_idletasks()    
    
def enterPeriod(entrybar):
    global report_period
    report_period = entrybar.get()
    if report_period:
        inputBtn_master.config(state='normal')
        inputBtn_period.config(state='disabled')
        entrybar.config(state='disabled')
        statusBar.config(text='Console Ready!\nSelect the relevant file/paths and click "Fill and Output Dashboard"')
    else:
        inputBtn_master.config(state='disabled')
    root.update_idletasks()
        
def getTemplateFile(inputLabel):
    global template_path
    file = askopenfilename(title='Select a data file',filetypes=[("Excel", "*.xlsx")])
    template_path = file
    inputLabel.config(text=os.path.basename(file))
    if os.path.isfile(template_path):
        inputBtn_files.config(state='normal')
        checkbox_files.config(state='normal')
    else:
        inputBtn_files.config(state='disabled')
        checkbox_files.config(state='disabled')
    root.update_idletasks()

def getFilesDirectory(inputLabel):
    global files_path
    path = askdirectory(title='Select directory')
    files_path = path
    inputLabel.config(text=path)
    if os.path.isdir(files_path):
        inputBtn_output.config(state='normal')
    else:
        inputBtn_output.config(state='disabled')
    root.update_idletasks()    
    
def getOutputDirectory(inputLabel):
    global output_path
    path = askdirectory(title='Select directory')
    output_path = path
    inputLabel.config(text=path)
    if os.path.isdir(output_path):
        startBtn.config(state='normal')
    else:
        startBtn.config(state='disabled')
    root.update_idletasks()

In [80]:
#GUI Console
Title ='Compliance Caseload - MO Dashboard Tool'
h=30
w=10
root = tk.Tk()
root.title(Title)
root.geometry("600x450")
root.resizable(0, 0) 

#Instructions Bar
tk.Label(root, text = " Please peform the steps below:", bd=1, relief='sunken', 
         anchor='w', font = ('Helvetica', 10, 'bold')).place(x=5, y=5, width = 590, height =h)

#Reporting Period Entry Bar & Button
tk.Label(root, text = "Please enter the reporting period (eg. 31 May 2021)").place(x=5, y=h+5, width =270, height =30)
entrybar_period = tk.Entry(root, width = 50, borderwidth = 5)
entrybar_period.place(x=70, y=h+30, width =525, height =25)
entrybar_period.insert(0, '')

inputBtn_period =tk.Button(root, text ="Enter", command=lambda:enterPeriod(entrybar_period))
inputBtn_period.place(x=5, y=h+30, width = 60, height =25)

#Master template Select Button & Display
tk.Label(root, text = "Please select the master template file...").place(x=-17, y=h+60, width =250, height =30)
inputLabel_master =tk.Label(root, text = "", bd=1, relief='sunken', anchor='w')
inputLabel_master.place(x=70, y=h+90, width =525, height =25)

inputBtn_master =tk.Button(root, text ="Select", command=lambda:getTemplateFile(inputLabel_master), state='disabled')
inputBtn_master.place(x=5, y=h+90, width = 60, height =25)

#Input Files (raw data) Select Button & Display
tk.Label(root, text = "Please select the files path...").place(x=-1, y=h+120, width =160, height =30)
inputLabel_files =tk.Label(root, text = "", bd=1, relief='sunken', anchor='w')
inputLabel_files.place(x=70, y=h+150, width =525, height =25)

inputBtn_files =tk.Button(root, text ="Select", command=lambda:getFilesDirectory(inputLabel_files), state='disabled')
inputBtn_files.place(x=5, y=h+150, width = 60, height =25)

var = tk.IntVar()
checkbox_files = tk.Checkbutton(root, text = 'Search raw files\nrecursively', variable = var, state='disabled')
checkbox_files.place(x=460, y=h+270)

#Output Path Select Button & Display
tk.Label(root, text = "Please select the output path...").place(x=5, y=h+180, width =160, height =30)
inputLabel_output =tk.Label(root, text = "", bd=1, relief='sunken', anchor='w')
inputLabel_output.place(x=70, y=h+210, width =525, height =25)

inputBtn_output =tk.Button(root, text ="Select", command=lambda:getOutputDirectory(inputLabel_output), state='disabled')
inputBtn_output.place(x=5, y=h+210, width = 60, height =25)

#Start (Fill & Export automation) Button
startBtn =tk.Button(root, text ="Fill and Output Dashboard", font = ('Helvetica', 12, 'bold'), fg = 'green', 
                    state='disabled', command=load_and_fill_workbook)
startBtn.place(x=w+20, y=h+260, width = 400, height =65)

#Status Bar
tk.Label(root, text = "Status Bar", font = ('Helvetica', 9, 'italic')).place(x=-263, y=360, width = 590, height =25)

statusBar = tk.Label(root, text = 'Console Ready!\nEnter the Reporting Period.', bd=1, relief='sunken')
statusBar.place(x=5, y= 382, width =590, height = h+30)


root.mainloop()